In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas import DataFrame

In [10]:
print('開始する都道府県コードを入力してください')
start_pref = input()
print(f'開始する都道府県コード : {start_pref}')

print('終了する都道府県コードを入力してください')
finish_pref = input()
print(f'終了する都道府県コー : {finish_pref}')


#工事費用
cost_df = DataFrame(pd.read_csv('Y:/機械学習/step8_1/prefecture_cost.csv', encoding='shift_jis'))
pref = ['北海道',
        '青森',
        '岩手',
        '宮城',
        '秋田',
        '山形',
        '福島',
        '茨城',
        '栃木',
        '群馬',
        '埼玉',
        '千葉',
        '東京都',
        '神奈川',
        '新潟',
        '富山',
        '石川',
        '福井',
        '山梨',
        '長野',
        '岐阜',
        '静岡',
        '愛知',
        '三重',
        '滋賀',
        '京都',
        '大阪',
        '兵庫',
        '奈良',
        '和歌山',
        '鳥取',
        '島根',
        '岡山',
        '広島',
        '山口',
        '徳島',
        '香川',
        '愛媛',
        '高知',
        '福岡',
        '佐賀',
        '長崎',
        '熊本',
        '大分',
        '宮崎',
        '鹿児島',
        '沖繩']
pref_df = DataFrame(pref, columns=['prefecture'])
cost_df = pd.merge(pref_df, cost_df, on='prefecture')

n = int(start_pref)
while n <= int(finish_pref):
    df = pd.read_csv(f'Y:/機械学習/step8_1/data/{n}_step8_1.csv')
    dmg_price_list = []

    #工事費用
    df['cost'] = np.where(df['estimated_structure'] == 0, cost_df['wood'][n] * 1000, cost_df['steel'][n] * 1000)
    #償却率
    df['dpc_rate'] = np.where(df['estimated_structure'] == 0, 0.031, 0.036)
    #築年数
    df['age'] = df['estimated_age'].map({1: 5, 2: 8, 3: 15.5, 4: 25.5, 5: 35.5, 6: 45.5, 7: 50})
    
    df['dmg_rate'] = np.where((df['A31_001'] == 11) | (df['A31_001'] == 21), np.where(df['floor'] == 0, 0.4, 0.35),
                          np.where((df['A31_001'] == 12) | (df['A31_001'] == 22), np.where(df['floor'] == 0, 0.6, 0.45),
                          np.where((df['A31_001'] == 13) | (df['A31_001'] == 23), np.where(df['floor'] == 0, 0.75, 0.5),
                          np.where(((df['A31_001'].between(14, 15)) | (df['A31_001'] >= 24)), np.where(df['floor'] == 0, 0.8, 0.55), 0.0))))
    
    df.apply(lambda row: dmg_value(row['cost'], row['total_area'], row['dpc_rate'], row['age'], row['dmg_rate']), axis=1)
    df = df.join(DataFrame(dmg_price_list, columns=['damage_price']))
    
    df.to_csv(f'Y:/機械学習/step8_1/data/output/{n}_step8_1_output.csv', index=False)
    n += 1

開始する都道府県コードを入力してください
開始する都道府県コード : 14
終了する都道府県コードを入力してください
終了する都道府県コー : 14


In [5]:
df.groupby('KEY_CODE', as_index=False)['damage_price'].sum()

,KEY_CODE,damage_price
0,141010180,9.604426e+08
1,141010490,2.556908e+09
2,141010560,1.623599e+09
3,141010570,1.237905e+09
4,141010580,1.910518e+09
...,...,...
1360,14366009001,2.109556e+08
1361,14366009002,5.511168e+05
1362,14366009003,9.168066e+07
1363,14366009004,9.471514e+07


In [13]:
df['KEY_CODE'].value_counts().sum()

265446